In [5]:
# !/usr/bin/env python
import pika
import json
credentials = pika.PlainCredentials('rabbitadmin','yulorei7')
connection = pika.BlockingConnection(pika.ConnectionParameters(
    '172.18.19.219',5672,'/',credentials))
channel = connection.channel()

# 声明queue
channel.queue_declare(queue='rpt_in_queue',durable=True)
import json
json_filename = ('../mongo/phone.json') 
_data=json.load(open(json_filename)) 

post_data=json.dumps({'data':_data,'cfg':{},'sid':'','retry_times':1})
# n RabbitMQ a message can never be sent directly to the queue, it always needs to go through an exchange.
channel.basic_publish(exchange='',
                      routing_key='rpt_in_queue',
                      body=post_data,
                      properties=pika.BasicProperties(
                          delivery_mode=2,  # make message persistent
                      )
                     )
print(" [x] Sent 'Hello World!'")
connection.close()

 [x] Sent 'Hello World!'


In [1]:
# !/usr/bin/env python
import pika
import json
credentials = pika.PlainCredentials('rabbitadmin','yulorei7')
connection = pika.BlockingConnection(pika.ConnectionParameters(
    '172.18.19.219',5672,'/',credentials))
channel = connection.channel()

exchange_topic1='report_in_topic_ex'
exchange_topic2='report_out_topic_ex'
queue_in_novip='rpt_in_novip'
queue_in_vip='rpt_in_vip'
queue_out_novip='rpt_out_novip'
queue_out_vip='rpt_out_vip'
DELAY_EXCHANGE = 'Dead_Letter_Ex'
REPORT_A_IN_QUEUE='delayed_a_in_queue'
REPORT_B_IN_QUEUE='delayed_b_in_queue'
REPORT_C_IN_QUEUE='delayed_c_in_queue'

# 创建交换机
channel.exchange_declare(exchange=exchange_topic1, exchange_type='topic', durable=True)
channel.queue_declare(queue=queue_in_novip, durable=True)
channel.queue_declare(queue=queue_in_vip, durable=True)
channel.queue_bind(queue=queue_in_novip, exchange=exchange_topic1, routing_key="report.novip")
channel.queue_bind(queue=queue_in_vip, exchange=exchange_topic1, routing_key="report.vip")

channel.exchange_declare(exchange=exchange_topic2, exchange_type='topic', durable=True)
channel.queue_declare(queue=queue_out_novip, durable=True)
channel.queue_declare(queue=queue_out_vip, durable=True)
channel.queue_bind(queue=queue_out_novip, exchange=exchange_topic2, routing_key="report.novip")
channel.queue_bind(queue=queue_out_vip, exchange=exchange_topic2, routing_key="report.vip")


arguments={'x-message-ttl' : 60*1000}
arguments.update({'x-dead-letter-exchange': DELAY_EXCHANGE})
channel.queue_declare(queue=REPORT_A_IN_QUEUE,durable=True,arguments=arguments)

arguments={'x-message-ttl' : 180*1000}
arguments.update({'x-dead-letter-exchange': DELAY_EXCHANGE})
channel.queue_declare(queue=REPORT_B_IN_QUEUE,durable=True,arguments=arguments)

arguments={'x-message-ttl' : 900*1000}
arguments.update({'x-dead-letter-exchange': DELAY_EXCHANGE})
channel.queue_declare(queue=REPORT_C_IN_QUEUE,durable=True,arguments=arguments)

channel.exchange_declare(exchange=DELAY_EXCHANGE, exchange_type='direct', durable=True)
channel.queue_bind(queue=queue_in_novip, exchange=DELAY_EXCHANGE, routing_key="report.novip")
channel.queue_bind(queue=queue_in_vip, exchange=DELAY_EXCHANGE, routing_key="report.vip")


ModuleNotFoundError: No module named 'pika'

In [ ]:
post_data=json.dumps({'data':'','cfg':{},'sid':'','retry_times':1})

# n RabbitMQ a message can never be sent directly to the queue, it always needs to go through an exchange.
channel.basic_publish(exchange=exchangeName,
                      routing_key='dlx',
                      body=post_data,
                      properties=pika.BasicProperties(
                          delivery_mode=2,  # make message persistent   
                    content_type='application/json',
                    content_encoding='UTF-8',
                    priority=0,
                    expiration="30000",
                      )
                     )
print(" [x] Sent 'Hello World!'")
connection.close()